# **Presentación del Caso Forest Cover Type**

En la clase de hoy trabajaremos con un dataset **privado** utilizado para la **predicción del tipo de árbool** según las características ingresadas en el dataset. Los datos son de caracter **geológico** y han sido extraídos en la Roosevelt National Forest ubicado en Colorado, Estados Unidos.

Se ha recopilado un total de 54 variables predictoras de tipo categórica y numérico, sin embargo la variable objetivo (target) es la columna **Cover Type** cuyo tipo de dato es **numérico**.

# **Comprensión de los datos**

In [0]:
#Crear una variable url para asignar la ruta url del archivo csv de entrenamiento guardado en mi github
url = 'https://raw.githubusercontent.com/javalillo13689/tesst/master/train.csv'

#Importaremos el url de nuestra data para hacer el test.
url_2 ='https://raw.githubusercontent.com/javalillo13689/tesst/master/test.csv'

#Importar la librería pandas para trabajar con dataframes y guardar esta libreria como una variable pd
import pandas as pd
import numpy as np
import seaborn as sns

#Crear una variable train para guardar la data de entrenamiento a partir del archivo csv obteniendo así un dataframe
#El índice del dataframe serán los valores que aparecen en la columna Id del archivo csv
train = pd.read_csv(url, index_col='Id')
test = pd.read_csv(url_2, index_col='Id')

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

In [0]:
train.head(2)

Verificamos el total de filas y las columnas

In [0]:
train.shape, test.shape

Comprobamos efectivamente que contamos con **54 variables predictoras**. Recuerda que **train** cuenta con **una columna adicional: CoverType**

# **Preprocesamiento de los datos**

##Verificación de nulos y/o outliers

Comprobamos si queda algún nulo en nuestra data

In [0]:
def valores_perdidos (tabla): 
        total_nulos = tabla.isnull().sum()
        porcentaje_nulos = 100 * tabla.isnull().sum()/len(tabla)
        tabla_nulos = pd.concat([total_nulos, porcentaje_nulos], axis=1)
        tabla_nulos_nombres = tabla_nulos.rename(columns = {0 : 'total_nulos', 1 : '% nulos'})
        return tabla_nulos_nombres [tabla_nulos_nombres['total_nulos'] != 0].sort_values('total_nulos', ascending=False)

##**Preprocesamiento de las variables o Feature Engineering**

**Añadimos una variable importante**

Debido a que la distancia **diagonal** es también importante a partir de las distancias horizontal y vertical a la fuente más cercana de agua, lo calculamos a partir de una **función** que definimos:

In [0]:
#Crear una variable list_1 para asignar una lista con los nombres de las columnas descritas
list_1=['Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology']

In [0]:
#Importar la libreria math para trabajar con funciones matemáticas
import math

#Definimos una funcion que denominamos pitagoras con dos parámetros que sacará la raíz cuadrada de la suma de los cuadrados de ambos parámetros
def pitagoras (a, b): 
    return math.sqrt(a**2 + b**2)

#Agregamos la columna pitagoras al dataframe train_pitagoras_distance1 donde guardar el resultado de ejecutar la función pitagoras fila por fila
train['pitagoras'] = train.apply(lambda row : pitagoras(row['Horizontal_Distance_To_Hydrology'],row['Vertical_Distance_To_Hydrology']), axis = 1)
#Tambien transformamos el test
test['pitagoras'] = test.apply(lambda row : pitagoras(row['Horizontal_Distance_To_Hydrology'],row['Vertical_Distance_To_Hydrology']),axis=1)

**Añadimos otra variable importante**

Debido a que la sombra **promedio** es también importante a partir de las sombras en distintas horas del día, lo calculamos con el método **mean**

In [0]:
list_2=['Hillshade_9am','Hillshade_Noon','Hillshade_3pm']
#Voy a sacar la media de las sombras de estas tres variables
train['hillmean'] = train[list_2].mean(axis=1) 
test['hillmean'] = test[list_2].mean(axis=1)

In [0]:
train.head(3)

# **Modelamiento**

En esta sesión utilizamos los siguientes **modelos de boosting**:

Los modelos que ejecutaremos serán:

1.   AdaBoost
2.   XGBoost
2.   LightGBM

In [0]:
#Crear una variable X para guardar el dataframe train
X = train

In [0]:
#Crear una variable y para guardar solo la columna Cover_Type de la variable X porque esta variable es el objetivo de la predicción
y = X['Cover_Type']

#Eliminar la columna Cover_Type de la variable X para solo conservar las variables predictoras.
#El parámetro inplace=True permite actualizar la variable X
X.drop(['Cover_Type'], axis=1, inplace=True)

In [0]:
#Importamos la función train_test_split para poder usar model_selection
from sklearn.model_selection import train_test_split

#Usando train_test_split obtenemos las muestras de entrenamiento (train) y test a partir de X tomando el 60% y 40% respectivamente
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state=1)

### Iniciamos con un modelo Random Forest para establecer una línea base

In [0]:
#Para tunear los hiperparámetros necesitamos importar GridSearchCV
from sklearn.model_selection import GridSearchCV

#Importamos la librería sklearn y model_selection para utilizar cross_val_score
from sklearn.model_selection import cross_val_score


In [0]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100)

#Especificamos los parámetros en los que queremos buscar lo óptimo
params_rf = {'max_depth' : [6,11,12,13], 'criterion' : ['gini','entropy'] }

#Hacemos la búsqueda
searcher_rf = GridSearchCV(rf_clf, params_rf)

searcher_rf.fit(X_train,y_train)

In [0]:
searcher_rf.best_params_

In [0]:
rf_clf_ = searcher_rf.best_estimator_
cross_val_score(rf_clf_, X_train, y_train, cv=5)

In [0]:
rf_clf_.score(X_test,y_test)

### AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

#En este caso no tunaeremos nuestro adaboost
ab_clf = AdaBoostClassifier(learning_rate=1,
                            random_state=337,
                            base_estimator=DecisionTreeClassifier(max_depth=10))

ab_clf.fit(X_train,y_train)

cross_val_score(ab_clf, X_train, y_train, cv=5)

In [0]:
ab_clf.score(X_test,y_test)

In [0]:
importances = ab_clf.feature_importances_
cols = X_train.columns
plt.figure(figsize=(8,12))
indices = np.argsort(importances)
plt.title('Feature importances con adaboost')
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)),[cols[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

### XGBOOST 

In [0]:
#Importamos la librería xgboost para utilizar XGBClassifier utilizado en la predicción de clases (variables categóricas)
#En este ejemplo la clase de la variable Category_Type
from xgboost import XGBClassifier

#Creamos la variable xgb_c para guardar XGBClassifier
xgb_c = XGBClassifier(learning_rate = 0.1,
                      n_estimators = 500, 
                      early_stopping_rounds = 10, 
                      max_depth = 10,
                      eval_metric = 'logloss', 
                      eval_set =[(X_test,y_test)] )


In [0]:
#Utilizamos los parámetros del modelo xgb_c para utilizar las muestras de entrenamiento
#El parámetro early_stopping_rounds indica el límite de iteraciones del modelo
#El parámetro eval_set indica una referencia para comparar los resultados de cada iteración del modelo
xgb_c.fit(X_train, y_train)

In [0]:
#Obtenemos las predicciones utilizando el parámetro predict del modelo xgb_c a partir del dataframe train4
#Creamos la variable predictions para guardar las predicciones
predictions = xgb_c.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(predictions,y_test)

In [0]:
#Importamos la librería sklearn y model_selection para utilizar cross_val_score
from sklearn.model_selection import cross_val_score

#Usando cross_val_score iteramos el modelo xgb_c a partir de las muestras de entrenamiento
#El parámetro cv hace referencia a las particiones de las muestras de entrenamiento para las iteraciones
cross_val_score(xgb_c, X_train, y_train, cv=5)

### LightGBM

In [0]:
import lightgbm as lgb
from sklearn.model_selection import KFold

d_train = lgb.Dataset(X_train, label=y_train)
folds = KFold(5, random_state=1)


params = {'task': 'train','boosting_type': 'gbdt','metric': { 'auc'}, "max_depth":2,
              "num_leaves":3,'learning_rate': 0.075,'feature_fraction': 0.9,
              'bagging_fraction': 0.9,'verbose': 1}


lgb_clf = lgb.train(params, d_train, 100)

results = lgb.cv(params, d_train, folds=folds.split(X_train), num_boost_round=1000, 
                 early_stopping_rounds=100)

lgb_pred = lgb_clf.predict(X_test)

In [0]:
lgb_pred